# Right Left up down 1200

## Setup

In [ ]:
import os
from pathlib import Path

import tensorflow as tf
import keras
import pandas as pd

from frame_generator import FrameGenerator
from model import create_model

In [ ]:
DATA_PATH = "data_7_right_left_up_down_1200"
NOTEBOOK_NAME = "3_2d_plus_1_rlud_1200_sp7"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

# PARAMS
n_frames = 36
frame_step = 1
batch_size = 8
HEIGHT = 224
WIDTH = 224

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

## Preprocess video data

In [ ]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        index_df=index_df,
        height=HEIGHT,
        width=WIDTH,
        frame_step=frame_step,
        training=True
    ),
    output_signature = output_signature
)
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['val'],
        n_frames=n_frames,
        height=HEIGHT,
        width=WIDTH,
        index_df=index_df,
        frame_step=frame_step,
        training=False
    ),
    output_signature = output_signature
)
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['test'],
        n_frames=n_frames,
        height=HEIGHT,
        width=WIDTH,
        index_df=index_df,
        frame_step=frame_step,
        training=False
    ),
    output_signature = output_signature
)

test_ds = test_ds.batch(batch_size)

## Model Creation

In [ ]:
model = create_model(n_frames=n_frames, height=HEIGHT, width=WIDTH)

## Build Model

In [ ]:
frames, label = next(iter(train_ds))

In [ ]:
model.build(frames)

## Load the Model

In [ ]:
previous_runs = 0

In [ ]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

## Train the Model

In [ ]:
previously_run_epochs = 0

history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

## Analyse results

In [ ]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

for batch in test_ds:
    x, y = batch
    true_labels.extend(y.numpy())
    preds = model.predict(x)
    preds = softmax(preds, axis=1)
    preds = np.argmax(preds, axis=1)
    predictions.extend(preds)

true_labels = np.array(true_labels)
predictions = np.array(predictions)

In [ ]:
fg = FrameGenerator(
    subset_paths['test'],
    n_frames=n_frames,
    index_df=index_df,
    height=HEIGHT,
    width=WIDTH,
    frame_step=frame_step,
    training=False
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

true_labels = np.array(true_labels)
predictions = np.array(predictions)

accuracy = accuracy_score(true_labels, predictions)

precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")